# Simulation of a single 20-bead linear chain

In [1]:
import unittest
import copy
import math
import pandas as pd

script="""
MonteCarlo
Configuration cubic_box_length 30 \
              particle_type0 /feasst/plugin/chain/forcefield/chain20.fstprt
Potential Model LennardJones VisitModel VisitModelIntra intra_cut 1
ThermoParams beta 1 chemical_potential0 1
Metropolis
TrialAdd particle_type 0
Run until_num_particles 1
RemoveTrial name TrialAdd
TrialPivot weight 1 tunable_param 20 max_length 8
TrialCrankshaft weight 1 tunable_param 20 max_length 8
TrialGrowFile file_name grow_chain20.txt
set_variable trials_per 1e4
Log trials_per trials_per file_name chain.txt
Movie trials_per trials_per file_name chain.xyz
Energy trials_per_write trials_per file_name chain_en.txt
CheckEnergy trials_per trials_per tolerance 1e-10
Tune
Run num_trials 1e3
EndToEndDistance file_name end_to_end.txt
RadiusOfGyration file_name rg.txt
Run num_trials 1e5
"""

# create grow_chain20.txt
unit_grows = list()
grows=list()
length=20
max_length=19
for i in range(max_length):
     unit_grows.append("bond true mobile_site "+str(i)+" anchor_site "+str(i+1)+" num_steps 4\n")
     if i == 0:
         grows.append(unit_grows[-1])
     else:
         grows.append(unit_grows[-1] + copy.deepcopy(grows[-1]))
unit_grows = list()
for i in range(max_length):
    unit_grows.append("bond true mobile_site "+str(length-i-1)+" anchor_site "+str(length-i-2)+'\n')
    if i == 0:
        grows.append(unit_grows[-1])
    else:
        grows.append(unit_grows[-1] + copy.deepcopy(grows[-1]))
for ind, grow in enumerate(grows):
    grows[ind] = "weight "+str(1./len(grow))+" "+grows[ind]
    grows[ind] = "particle_type 0 " + grows[ind]
with open('grow_chain20.txt', 'w') as file:
    file.write('TrialGrowFile\n\n')
    for grow in grows:
        file.write(grow+'\n')

with open('script.txt', 'w') as file: file.write(script)
import subprocess
syscode = subprocess.call("../../../build/bin/fst < script.txt > script.log", shell=True, executable='/bin/bash')
with open('script.log', 'r') as file: print(file.read(), '\n', 'exit:', syscode)

# FEASST version: v0.19.0-8-gceffc51431-hwh/tminit
Configuration cubic_box_length 30 particle_type0 /home/hwh/feasst/plugin/chain/forcefield/chain20.fstprt  
Potential Model LennardJones VisitModel VisitModelIntra intra_cut 1  
ThermoParams beta 1 chemical_potential0 1  
Metropolis  
TrialAdd particle_type 0  
Run until_num_particles 1  
# initializing random number generator with seed: 1654262238
RemoveTrial name TrialAdd  
TrialPivot max_length 8 tunable_param 20 weight 1  
TrialCrankshaft max_length 8 tunable_param 20 weight 1  
TrialGrowFile file_name grow_chain20.txt  
Log file_name chain.txt trials_per 1e4  
Movie file_name chain.xyz trials_per 1e4  
Energy file_name chain_en.txt trials_per_write 1e4  
CheckEnergy tolerance 1e-10 trials_per 1e4  
Tune  
Run num_trials 1e3  
EndToEndDistance file_name end_to_end.txt  
RadiusOfGyration file_name rg.txt  
Run num_trials 1e5  
 
 exit: 0


In [2]:
class TestChain20RG(unittest.TestCase):
    def test_rg_en(self):
        rg=pd.read_csv('rg.txt')
        self.assertAlmostEqual(2.45, rg['average'][0], delta=4*rg['block_stdev'][0])
        end_to_end=pd.read_csv('end_to_end.txt')
        self.assertAlmostEqual(2.92, end_to_end['average'][0], delta=4*end_to_end['block_stdev'][0])
        en=pd.read_csv('chain_en.txt')
        self.assertAlmostEqual(-29.4, en['average'][0], delta=4*en['block_stdev'][0])
unittest.main(argv=[''], verbosity=2, exit=False)

test_rg_en (__main__.TestChain20RG) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.009s

OK


Did this tutorial work as expected? Did you find any inconsistencies or have any comments? Please [contact](../../../CONTACT.rst) us. Any feedback is appreciated!